In [55]:
gcs_bucket="mickael-dm-stt"
gcs_transcripts_path="transcripts"
gcs_ground_truth_path="ground_truth"

In [56]:
import re

model_folders=!(gsutil ls gs://{gcs_bucket}/{gcs_transcripts_path})
models=[]

for folder in model_folders:
    #regex="gs://"+gcs_bucket+"/"+"transcripts"+"/(.*)/$"
    regex=".*/(.*)/$"
    model=re.search(regex,folder).group(1)
    models.append(model)

print(models)

['chirp', 'long', 'whisper_base', 'whisper_large', 'whisper_medium', 'whisper_small', 'whisper_tiny']


In [57]:
import re
from google.cloud import storage

# Instantiates a Cloud Storage client
storage_client = storage.Client()
bucket = storage_client.bucket(gcs_bucket)
videos=[]

ground_truth_uris=!(gsutil ls gs://{gcs_bucket}/{gcs_ground_truth_path})
for uri in ground_truth_uris:
    regex="gs://"+gcs_bucket+"/(.*)/(\w+).txt$"
    path,video_id=re.search(regex,uri).group(1,2)
    print(f"Video ID: {video_id}")

    ground_truth_blob=bucket.blob(path+"/"+video_id+".txt")
    ground_truth=ground_truth_blob.download_as_text()
    print(f"Ground truth: {ground_truth}")
    
    video={
        "id": video_id,
        "ground_truth": ground_truth,
        "transcripts": []
    }
    
    for model in models:
        transcript_blob=bucket.blob(gcs_transcripts_path+"/"+model+"/txt/"+video_id+".txt")
        transcript=transcript_blob.download_as_text()
        print(f"Transcript for {model}: {transcript}")
        video["transcripts"].append([model, transcript])
    videos.append(video)

Video ID: x58y7eg
Ground truth: I remember getting a detention or suspended for skateboarding in the hallways my principal saying if you think you're going to grow up and be able to skateboard through the halls of your office you got another thing coming.
Um, I built a half pipe in my office a few years ago just so I can skateboard while I worked.
My name is Casey Neistat and I'm a filmmaker and a youtuber and a business owner and a dad, I wear a lot of hats.
I was born in South-Eastern Connecticut in a town called Gales Ferry which is like a little kind of nothing town.
I got in a lot of trouble as a kid, like a lot of trouble in grammar school just for being bad.
It wasn't a lack of respect for authority, it was like I disagreed with the authority because I was being told things I just didn't agree with.
And if I didn't agree with it, why should I subscribe to it?
Like I would describe so much of what I experienced, and so much of the guidance that I was given right up until my teena

In [58]:
import os
from simple_wer_v2 import SimpleWER
import pandas as pd

video_ids=[]
wer_scores=[]

for video in videos:
    print(f"Video: {video['id']}")
    #video["wer"]=[]
    video_ids.append(video["id"])
    wer_score={}
    for transcript in video["transcripts"]:
        #print(f"Transcript for {transcript[0]}: {transcript[1]}")
        werClient=SimpleWER()
        werClient.AddHypRef(transcript[1], video["ground_truth"])
        wer=werClient.GetWER()
        #video["wer"].append([transcript[0],wer_score])
        wer_score[transcript[0]]=wer
        print(f"WER score for {transcript[0]}: {wer}")
        html="<body><html><div>"+werClient.aligned_htmls[0]+"</div></body></html>"
        
        if not os.path.exists(f"wer_diagnosis/{transcript[0]}/"):
            os.makedirs(f"wer_diagnosis/{transcript[0]}/")
        file=open("wer_diagnosis/"+transcript[0]+"/"+video['id']+".html","w")
        file.write(html)
        file.close()
    wer_scores.append(wer_score)
print(wer_scores)

df=pd.DataFrame(wer_scores, index=video_ids)
df

Video: x58y7eg
WER score for chirp: 3.913491246138002
WER score for long: 9.371781668383111
WER score for whisper_base: 8.650875386199795
WER score for whisper_large: 9.88671472708548
WER score for whisper_medium: 5.355303810504634
WER score for whisper_small: 5.8702368692070035
WER score for whisper_tiny: 12.976313079299691
Video: x8mmzj2
WER score for chirp: 8.55188141391106
WER score for long: 16.419612314709237
WER score for whisper_base: 24.401368301026224
WER score for whisper_large: 16.76168757126568
WER score for whisper_medium: 24.857468643101484
WER score for whisper_small: 19.27023945267959
WER score for whisper_tiny: 33.75142531356899
Video: x8mv7he
WER score for chirp: 3.7735849056603774
WER score for long: 8.355795148247978
WER score for whisper_base: 16.172506738544474
WER score for whisper_large: 4.5822102425876015
WER score for whisper_medium: 6.199460916442049
WER score for whisper_small: 12.398921832884097
WER score for whisper_tiny: 25.87601078167116
Video: x8n0jf1


,chirp,long,whisper_base,whisper_large,whisper_medium,whisper_small,whisper_tiny
x58y7eg,3.913491,9.371782,8.650875,9.886715,5.355304,5.870237,12.976313
x8mmzj2,8.551881,16.419612,24.401368,16.761688,24.857469,19.270239,33.751425
x8mv7he,3.773585,8.355795,16.172507,4.582210,6.199461,12.398922,25.876011
x8n0jf1,8.272251,13.403141,13.298429,13.926702,26.178010,11.413613,20.314136
x8n29uv,8.750000,15.000000,12.500000,7.500000,8.750000,10.625000,13.125000
x8n38du,8.227848,10.126582,15.189873,9.493671,9.493671,12.658228,29.113924
x8n3e82,27.500000,35.000000,52.500000,25.000000,22.500000,40.000000,62.500000
x8n3jhn,14.159292,11.061947,28.761062,7.964602,18.584071,18.584071,46.902655
x8n3qv2,0.847458,1.694915,7.627119,0.847458,0.847458,2.542373,12.711864
x8n3ssu,0.000000,0.000000,3300.000000,2800.000000,23000.000000,200.000000,10400.000000
